# 분류 시스템을 이용해서 자동으로 태그 달아주기
---
개념

    옷을 넣었을 때, 해당되는 카테고리/태그 출력해주기
    
카테고리란?

    무신사의 major/middle/minor 분류
    
태그란?

    사용자의 임의로 붙여진 #문자열
    
출력할 카테고리의 범위

1. 대분류 단위  
    장점 : 남녀간 형태차이가 확실한 옷은 잘 분류  
    단점 : 세부적인 추천이 불가능함. 티셔츠같은 경우는 남자 여자 구분이 쉽지않음
2. 중분류 단위  
    장점 : 가장 적절한 분류 카테고리  
    단점 : 무신사의 특성상, 중분류 밑에서도 옷의 종류가 굉장히 많이 차이남
3. 소분류 단위  
    장점 : 섬세한 추천이 가능  
    단점 : 카테고리 수가 매우 늘어나 난이도 상승,  
    같은 소분류라도 다른 대분류 하에 있을 수 있는데 이를 어떻게 처리하지
    
분류가 비교적 정확히 되는 속성 : color/shape  
분류가 비교적 정확히 되지 않는 속성 : texture/small pattern

## CNN-only
원리

    Classification 방식 도입
    
    train : 이미지-카테고리 데이터셋으로, CNN을 통해 지도학습시킴
    test : 새로운 옷이 들어온 경우 해당 모델에 집어넣고 나오는 카테고리를 사용
    model : 여기에서 사용된 모델이 추후 feature extraction에도 사용됨
    
제공할 데이터

    이미지(필수)

모델 접근법
1. 중->소 순으로 넘어가기      
    아무리 생각해도 one-for-all 모델은 존재하지 않음  
    특히 분류를 소분류단위로 할 경우는 더더욱  
    따라서, 중분류 단위로 모델을 만들고, 카테고리
  
2. **소분류로 한번에 분류하기**  
    남/녀 구분없이 그냥 소분류로 때려버리는것도 괜찮을듯  
    정확하게 소분류를 구분하지 못하더라도, 사용자에게 컨펌받아 수정하면 되니까!

Multi Category 분류 방법(Blue/T-shirt 등)  
https://github.com/KerasKorea/KEKOxTutorial/blob/master/35_케라스를%20이용한%20다중%20라벨%20분류.md
    


데이터 준비 - minor/숫자 매핑 (3중 조인으로 해결해서 필요없을줄 알았는데 one-hot encoding에 필요함)

In [1]:
#경로 설정용
import os

os.chdir('..')

In [2]:
#DB 연결
import json
import pymysql
import numpy as np
import pandas as pd

#데이터 url/fs 전처리용
import requests
from PIL import Image
from io import BytesIO
from tensorflow.keras.utils import Sequence
from skimage.io import imread
from skimage.transform import resize

#모델 작성용
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.backend import clear_session
from tensorflow.keras.applications import resnet50, vgg16, vgg19, inception_resnet_v2,inception_v3, densenet
from tensorflow.keras.applications import mobilenet,mobilenet_v2, nasnet, xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D,MaxPooling2D,GlobalAveragePooling2D, Dropout

In [3]:
def connect_db(dbinfo_path) :
    with open(dbinfo_path) as jsonfile :
        dbinfo = json.load(jsonfile)

    connection = pymysql.connect(host=dbinfo['host'],
                         port=dbinfo['port'],
                         user=dbinfo['user'], 
                         passwd=dbinfo['passwd'],
                         db=dbinfo['db'],
                         charset=dbinfo['charset'])
    
    return connection

In [4]:
#DB로부터 minor 카테고리 정보 획득
db = connect_db('dbinfo-temp.json')
cur = db.cursor()

sql = 'SELECT MIN(id),minor FROM CLOTHES_CLASS GROUP BY minor HAVING minor != "기능성 상의";' # \으로 개행해도 SQL 정상 동작 확인
cur.execute(sql)

40

In [5]:
minor_list = []
for index,minor in cur :
    minor_list.append([minor,index])

In [6]:
minor_dict = np.array(minor_list)

In [7]:
minor_dict

array([['겨울 기타 코트', '61'],
       ['겨울 싱글 코트', '116'],
       ['기타 바지', '83'],
       ['기타 상의', '96'],
       ['기타 아우터', '52'],
       ['긴팔 티셔츠', '46'],
       ['나일론/코치  재킷', '108'],
       ['니트/스웨터/카디건', '93'],
       ['데님 팬츠', '71'],
       ['레깅스', '81'],
       ['레더/라이더스 재킷', '100'],
       ['롱 스커트', '88'],
       ['롱 패딩/롱 헤비 아우터', '63'],
       ['맥시 원피스', '58'],
       ['맨투맨/스웨트셔츠', '44'],
       ['미니 스커트', '85'],
       ['미니 원피스', '55'],
       ['미디 스커트', '87'],
       ['미디 원피스', '57'],
       ['민소매 티셔츠', '50'],
       ['반팔 티셔츠', '42'],
       ['베스트', '68'],
       ['블루종/MA-1', '98'],
       ['사파리/헌팅 재킷', '69'],
       ['셔츠/블라우스', '89'],
       ['숏 패딩/숏 헤비 아우터', '65'],
       ['숏 팬츠', '79'],
       ['수트 팬츠/슬랙스', '74'],
       ['수트/블레이저 재킷', '104'],
       ['스타디움 재킷', '112'],
       ['아노락 재킷', '106'],
       ['점프수트', '59'],
       ['코튼 팬츠', '73'],
       ['트러커 재킷', '102'],
       ['트레이닝 재킷', '110'],
       ['트레이닝/조거 팬츠', '77'],
       ['패딩 베스트', '53'],
       ['피케/카라 티셔츠', '91'],
 

In [8]:
# one-hot encoding by pandas
minor_df = pd.DataFrame(minor_dict)
minor_onehot = pd.get_dummies(minor_df[0])

In [9]:
minor_onehot

,겨울 기타 코트,겨울 싱글 코트,기타 바지,기타 상의,기타 아우터,긴팔 티셔츠,나일론/코치 재킷,니트/스웨터/카디건,데님 팬츠,레깅스,...,아노락 재킷,점프수트,코튼 팬츠,트러커 재킷,트레이닝 재킷,트레이닝/조거 팬츠,패딩 베스트,피케/카라 티셔츠,환절기 코트,후드 스웨트셔츠/후드 집업
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


데이터 준비 - DB에서 url 긁어오기

---

DB에서 url긁은 후 http: 붙여 전처리  
train/val/test set 생성  
train:val:test = 0.72:0.18:0.1  

In [ ]:
"""
1. DB 호출. url만 존재하는 상태
2. Train/Test 임의 분할해서 x_train/y_train x_test/y_test
3. y label을 80-size vector로 변환
4. batch generator 만들어서 url에 있는 파일 불러오기
5. array로 만들어서 batch화
6. keras로 훈련 가즈아
"""

In [9]:
# 1. DB 호출
sql = 'SELECT CLOTHES.id, CLOTHES.img, CLOTHES_CLASS.minor \
FROM CLOTHES \
INNER JOIN CLOTHES_AND_CLOTHES_CLASS ON CLOTHES.id=CLOTHES_AND_CLOTHES_CLASS.clothes_id \
INNER JOIN CLOTHES_CLASS ON CLOTHES_AND_CLOTHES_CLASS.clothes_class_id = CLOTHES_CLASS.id \
WHERE CLOTHES_AND_CLOTHES_CLASS.clothes_class_id != 120 \
LIMIT 10000;' # \으로 개행해도 SQL 정상 동작 확인

cur.execute(sql)

10000

In [10]:
id_data = []
x_data = []
y_data = []
for id,url,minor in cur :
    id_data.append(id)
    x_data.append("http:"+url)
    y_data.append(minor)

In [11]:
#SQL로 불러와진 ID 순서
id_data

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 187,
 1308,
 2122,
 2123,
 2124,
 2125,
 2126,
 2127,
 2128,
 2130,
 2131,
 2132,
 2133,
 2134,
 2135,
 2136,
 2137,
 2138,
 2139,
 2140,
 2141,
 2142,
 2143,
 2144,
 2145,
 2146,
 2147,
 2148,
 2149,
 2150,
 2152,
 2153,
 2154,
 2155,
 2156,
 2157,
 2158,
 2159,
 2160,
 2161,
 2162,
 2164,
 2165,
 2166,
 2167,
 2168,
 2169,
 2170,
 2171,
 2172,
 2173,
 2175,
 2176,
 2178,
 2180,
 2181,
 2182,
 2183,
 2184,
 2185,
 2186,
 2187,
 2188,
 2189,
 4045,
 4047,
 4049,
 4050,
 4051,
 4052,
 4053,
 4054,
 4055,
 4056,
 4057,
 4058,
 4059,
 4060,
 4061,
 4062,
 4063,
 4064,
 4065,
 4066,
 4067,
 4068,
 4069,
 4070

In [12]:
x_data[:5] , y_data[:5]

(['http://image.msscdn.net/images/goods_img/20200416/1406534/1406534_1_500.jpg',
  'http://image.msscdn.net/images/goods_img/20200416/1406535/1406535_1_500.jpg',
  'http://image.msscdn.net/images/goods_img/20200416/1406536/1406536_1_500.jpg',
  'http://image.msscdn.net/images/goods_img/20200416/1406656/1406656_1_500.jpg',
  'http://image.msscdn.net/images/goods_img/20200319/1359987/1359987_1_500.jpg'],
 ['반팔 티셔츠', '반팔 티셔츠', '반팔 티셔츠', '반팔 티셔츠', '반팔 티셔츠'])

In [15]:
#2. Train/Test Split
from sklearn.model_selection import train_test_split

seed = 2233
x_temp,x_test = train_test_split(x_data,train_size=0.9,random_state=seed)
x_train,x_val = train_test_split(x_temp,train_size=0.8,random_state=seed)

y_temp,y_test = train_test_split(y_data,train_size=0.9,random_state=seed)
y_train,y_val = train_test_split(y_temp,train_size=0.8,random_state=seed)

In [46]:
#3. y label 40차원 one-hot-encoding으로 변환
y_train_oh = []
y_val_oh = []
y_test_oh = []

for minor in y_train :
    y_train_oh.append(minor_onehot[minor].to_numpy())
    
for minor in y_val :
    y_val_oh.append(minor_onehot[minor].to_numpy())
    
for minor in y_test :
    y_test_oh.append(minor_onehot[minor].to_numpy())

데이터 준비 - 이미지

---

1만개에 달하는 이미지 - 한번에 array로 RAM에 올릴 경우 OOM에러 발생가능성 있음  
-> 파일 경로만 저장해두고, 배치를 통해 순차적으로 image array로 전환

In [29]:
#3. batch generator 생성해서 리스트에서 64개 선정 후 get_image로 image array 생성
class My_Custom_Generator(Sequence) :
  def __init__(self, image_urls, labels, batch_size) :
    self.image_filenames = image_urls
    self.labels = labels
    self.batch_size = batch_size
 
  def __get_imgarray(self,url) :
    size=(224, 224)
    
    res = requests.get(url)
    image = BytesIO(res.content)
    image = Image.open(image)
    image = image.resize(size)

    image = np.array(image)
    return image

  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    return np.array([ self.__get_imgarray(file_name) for file_name in batch_x])/255.0, np.array(batch_y)

In [47]:
#generator 인스턴스 생성
batch_size = 64

my_training_batch_generator = My_Custom_Generator(x_train, y_train_oh, batch_size)
my_validation_batch_generator = My_Custom_Generator(x_val, y_val_oh, batch_size)

모델 생성하기

---
pretrained model 사용


In [17]:
#사용할 pretrained model 선택
def select_pre_model(name,weight,include_top=True) :
    name_dict = {'resnet50':resnet50.ResNet50,
                'vgg16':vgg16.VGG16,
                 'vgg19':vgg19.VGG19,
                 'inception_v2': inception_resnet_v2.InceptionResNetV2,
                 'inception_v3' : inception_v3.InceptionV3,
                 'densenet121':densenet.DenseNet121,
                 'densenet169':densenet.DenseNet169,
                 'densenet201':densenet.DenseNet201,
                 'mobilenet':mobilenet.MobileNet,
                 'mobilenet_v2': mobilenet_v2.MobileNetV2,
                 'nasnet' : nasnet.NASNetMobile,
                 'xception':xception.Xception
                }
    if top :
        return name_dict[name](weights=weight)
    else :
        return name_dict[name](weights=weight,include_top=False,input_shape=(224,224,3))

In [35]:
top= True # True/False : input/output layer 포함/미포함
name = 'vgg19'
pre_model = select_pre_model(name,'imagenet',include_top=top)

In [36]:
#pretrained 모델의 전체 모델 정보 확인
pre_model.summary()
print("전체 모델 depth는 :",len(pre_model.layers))

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [37]:
#pretrained 모델의 input/output layer 확인
pre_model.layers[0].input_shape,pre_model.layers[-1].output_shape

([(None, 224, 224, 3)], (None, 1000))

In [38]:
#레이어 freeze
#가장 뒷쪽 n개 레이어 제외하고 freeze
unfrozen_layer = -3
for layer in pre_model.layers[0:unfrozen_layer]:
    layer.trainable = False

In [39]:
#Top 여부에 따라 모델의 마지막 레이어 날리기
#top=True면 날리기, Top=False면 날리지 말기
last_layer = -2 

if top :
    model = Model(pre_model.inputs, pre_model.layers[last_layer].output)
else :
    model = Model(pre_model.inputs,pre_model.outputs)

In [40]:
#새 모델의 input/output layer 확인
model.layers[0].input_shape, model.layers[-1].output_shape

([(None, 224, 224, 3)], (None, 4096))

In [41]:
#현재 카테고리 개수 확인 40
num_category = len(minor_dict)

In [42]:
# 새 모델에 Avg Pooling과 Dense 추가
# output dense layer unit은 현재 카테고리 개수만큼으로 설정

new_model = Sequential()
new_model.add(model)
#new_model.add(GlobalAveragePooling2D())
new_model.add(Dense(num_category,activation="softmax"))

In [47]:
#과거 모델 지우기
clear_session()
del pre_model
del model

In [48]:
#새 모델 iput output shape 확인
print('Input Shape = {}'.format(new_model.layers[0].input_shape))
print('output Shape = {}'.format(new_model.layers[-1].output_shape))
new_model.summary()

Input Shape = (None, 224, 224, 3)
output Shape = (None, 40)
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 4096)              139570240 
_________________________________________________________________
dense_1 (Dense)              (None, 40)                163880    
Total params: 139,734,120
Trainable params: 119,709,736
Non-trainable params: 20,024,384
_________________________________________________________________


In [49]:
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

모델 학습

In [50]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "checkpoints/{name}"
checkpoint_path += "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

In [52]:
# 모델 학습 with generator
new_model.fit_generator(generator=my_training_batch_generator,
                   steps_per_epoch = int(len(x_train) // batch_size),
                   epochs =250,
                   verbose = 1,
                   callbacks = [cp_callback],
                   validation_data = my_validation_batch_generator,
                   validation_steps = int(len(x_val) // batch_size))

Epoch 1/250


ResourceExhaustedError: OOM when allocating tensor with shape[64,224,224,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

### 학습속도, 정확도 개선을 위한 수술

#### 데이터 준비과정의 문제

1. 이미지 url로 받지 말고, 그냥 파일시스템에서 받을까?  
->결과: 확실히 url로 받는 것 보다는 빠르다

In [11]:
#1 DB 호출
sql = 'SELECT CLOTHES.id, CLOTHES.img, CLOTHES_CLASS.minor \
FROM CLOTHES \
INNER JOIN CLOTHES_AND_CLOTHES_CLASS ON CLOTHES.id=CLOTHES_AND_CLOTHES_CLASS.clothes_id \
INNER JOIN CLOTHES_CLASS ON CLOTHES_AND_CLOTHES_CLASS.clothes_class_id = CLOTHES_CLASS.id \
WHERE CLOTHES_AND_CLOTHES_CLASS.clothes_class_id != 120 \
LIMIT 10000;' # \으로 개행해도 SQL 정상 동작 확인

cur.execute(sql)

10000

In [12]:
id_data = []
y_data = []
for id,url,minor in cur :
    id_data.append(id)
    y_data.append(minor)

In [13]:
#data/musinsa/1-10000.jpg
x_data = []
for i in id_data :
    x_data.append('data/musinsa/{}.jpg'.format(id))

In [14]:
#2 Train/Test Split
from sklearn.model_selection import train_test_split

seed = 2233
x_temp,x_test = train_test_split(x_data,train_size=0.9,random_state=seed)
x_train,x_val = train_test_split(x_temp,train_size=0.8,random_state=seed)

y_temp,y_test = train_test_split(y_data,train_size=0.9,random_state=seed)
y_train,y_val = train_test_split(y_temp,train_size=0.8,random_state=seed)

In [15]:
#3 y label 40차원 one-hot-encoding으로 변환
y_train_oh = []
y_val_oh = []
y_test_oh = []

for minor in y_train :
    y_train_oh.append(minor_onehot[minor].to_numpy())
    
for minor in y_val :
    y_val_oh.append(minor_onehot[minor].to_numpy())
    
for minor in y_test :
    y_test_oh.append(minor_onehot[minor].to_numpy())

In [16]:
#4 파일시스템 전용 batch generator 생성해서 리스트에서 64개 선정 후 get_image로 image array 생성
class My_Custom_Generator(Sequence) :
  def __init__(self, image_urls, labels, batch_size) :
    self.image_filenames = image_urls
    self.labels = labels
    self.batch_size = batch_size
 
  def __get_imgarray(self,url) :
    size=(224, 224)
    
    res = requests.get(url)
    image = BytesIO(res.content)
    image = Image.open(image)
    image = image.resize(size)

    image = np.array(image)
    return image

  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)

  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]

    return np.array([resize(imread(file_name), (224, 224, 3))
                   for file_name in batch_x])/255.0, np.array(batch_y)

In [17]:
#generator 인스턴스 생성
#batch_size=32로 하니 GPU(8GB)감당 가능한 수준
batch_size = 64

my_training_batch_generator = My_Custom_Generator(x_train, y_train_oh, batch_size)
my_validation_batch_generator = My_Custom_Generator(x_val, y_val_oh, batch_size)

2. 데이터를 Array가 아니라 다른 데이터포맷으로 넣으면 될까  
-> 가능한 데이터포맷 : tf dataset, pickle  
-> 방식 : path 리스트, one_hot list -> datasets에 저장 -> batch화 해서 불러오기

In [18]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224, 224))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [19]:
def load_image_tuple(x,y) :
    return load_image(x),y

In [20]:
#tf.dataset을 이용하는법 확인

image_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train_oh))
image_dataset = image_dataset.map( load_image_tuple, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(64)

3. 배치 대신 통으로 올려서 학습하면?  
->x_data를 통으로 다 image로 바꾸자

In [21]:
total_num = len(x_data)//2
x_data_half = np.array([resize(imread(file_name), (224, 224, 3)) for file_name in x_data[:total_num]])

KeyboardInterrupt: 

In [40]:
y_data_half = y_data[:total_num]

4. 카테고리 선택의 문제  
-> 너무 세부적으로 분류하려 하지 말고, 중분류단위로 하는것도 고려  
-> 소분류 중에서도 숫자가 많은 것들만 골라서 학습하면 어떨까

#### 모델 선정의 문제

1. pre-trained 모델에서 unfrozen 수를 줄이면(=freeze한 레이어의 수를 늘리면?)  
->  결과 : 빨라지긴 함 근데 정확도랑은 trade-off일수도?

2. 모델을 좀 더 간단한 걸 써야되나?  
-> 훈련이 짧고 빠른 커스텀 모델을 사용해보자  
-> Model 이용 Functional Subclassing 대신 Sequential로 바로 정의

In [22]:
num_category = len(minor_dict)

model = Sequential()

# 특징추출 CNN
# input = (224,224,3)
# output = (112,112,64)
model.add(Conv2D(filters=64, kernel_size=(3,3),padding="same",activation="selu",input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

# output = (56,56,128)
model.add(Conv2D(filters=128, kernel_size=(3,3),padding="same",activation="selu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

# output = (28,28,256)
model.add(Conv2D(filters=256, kernel_size=(3,3),padding="same",activation="selu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

# output = (14,14,512)
model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same",activation="selu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

# output = (7,7,512)
model.add(Conv2D(filters=512, kernel_size=(3,3),padding="same",activation="selu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))

# 판별 FCNN
model.add(tf.keras.layers.Flatten()) # Flatten()은 이미지를 일차원으로 바꿔줍니다.
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(num_category, activation='softmax'))

In [23]:
#datasets batch를 이용해 훈련
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [24]:
#10000개 7:2:1
model.fit(image_dataset,epochs=100)

Epoch 1/100
113/113 [==============================] - 69s 612ms/step - loss: 3.6211 - accuracy: 0.0343
Epoch 2/100
113/113 [==============================] - 64s 566ms/step - loss: 3.5869 - accuracy: 0.0349
Epoch 3/100
113/113 [==============================] - 64s 564ms/step - loss: 3.5782 - accuracy: 0.0358
Epoch 4/100
113/113 [==============================] - 64s 563ms/step - loss: 3.5739 - accuracy: 0.0318
Epoch 5/100
113/113 [==============================] - 64s 563ms/step - loss: 3.5726 - accuracy: 0.0350
Epoch 6/100
113/113 [==============================] - 64s 563ms/step - loss: 3.5725 - accuracy: 0.0321
Epoch 7/100
113/113 [==============================] - 64s 563ms/step - loss: 3.5710 - accuracy: 0.0319
Epoch 8/100
113/113 [==============================] - 64s 565ms/step - loss: 3.5702 - accuracy: 0.0346
Epoch 9/100
113/113 [==============================] - 64s 565ms/step - loss: 3.5692 - accuracy: 0.0322
Epoch 10/100
113/113 [==============================] - 64s 568m

KeyboardInterrupt: 

#### 학습 과정에서의 문제

1. 배치 제네레이터의 코드의 문제  
ValueError: could not broadcast input array from shape (224,224,3) into shape (224,224)  
-> 이는 url호출과정에서의 문제인가 아님 파일 자체의 문제인가  
-> 결과 : 파일시스템에서는 잘 동작, url 호출과정의 문제인듯


2. 매우 간단한 모델임에도 불구하고 OOM이 뜬다. 해결하자  
-> memory profile를 이용한 메모리 소모 확인  
-> gc.collect() 을 이용한 메모리 정리

## CNN-RNN 이용

제공할 데이터

    이미지(필수)
    제품명(넣으면 정확도 급격히 상승할 듯?) -> 그러나 자연어처리 과정이 추가적으로 필요
    제품설명 (No. 애초에 설명을 달기 귀찮아서 자동태그를 쓰는건데, 앞뒤가 안 맞다)
    
참고 문서  
https://medium.com/mighty-data-science-bootcamp/번역-머신러닝을-활용한-제품-카테고리-분류하기-cd75f11d588a